In [1]:
"""
Check Service Principal Permissions
Verify that your Service Principal has the required permissions
"""

from azure.identity import ClientSecretCredential, DefaultAzureCredential
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.storage import StorageManagementClient
from azure.mgmt.resource import ResourceManagementClient
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("="*60)
print("SERVICE PRINCIPAL PERMISSIONS CHECK")
print("="*60)

# Create credential
if os.getenv("AZURE_CLIENT_ID") and os.getenv("AZURE_CLIENT_SECRET") and os.getenv("AZURE_TENANT_ID"):
    credential = ClientSecretCredential(
        tenant_id=os.getenv("AZURE_TENANT_ID"),
        client_id=os.getenv("AZURE_CLIENT_ID"),
        client_secret=os.getenv("AZURE_CLIENT_SECRET")
    )
    print(f"✓ Using Service Principal: {os.getenv('AZURE_CLIENT_ID')}")
else:
    credential = DefaultAzureCredential()
    print("Using DefaultAzureCredential")

subscription_id = os.getenv("SUBSCRIPTION_ID")
resource_group = os.getenv("RESOURCE_GROUP")

print(f"  Subscription: {subscription_id}")
print(f"  Resource Group: {resource_group}")
print()

SERVICE PRINCIPAL PERMISSIONS CHECK
✓ Using Service Principal: e77fcc8e-5551-47b6-a600-1d5633c81e31
  Subscription: b8d6d487-0bd2-4773-b318-12ab763ed178
  Resource Group: strategicai-rg-uks-dev-01



In [2]:
# Initialize clients
auth_client = AuthorizationManagementClient(credential, subscription_id)
storage_client = StorageManagementClient(credential, subscription_id)
resource_client = ResourceManagementClient(credential, subscription_id)

In [3]:
# ==============================================================================
# Check Storage Account Access
# ==============================================================================
print("🔍 Checking Storage Account Access...")
print("-"*40)

try:
    # List storage accounts in the resource group
    storage_accounts = storage_client.storage_accounts.list_by_resource_group(resource_group)
    
    for account in storage_accounts:
        if "strategicaistuksdev02" in account.name:
            print(f"✓ Found storage account: {account.name}")
            
            # Try to get storage account keys (requires proper permissions)
            try:
                keys = storage_client.storage_accounts.list_keys(
                    resource_group, 
                    account.name
                )
                print(f"  ✓ Can list storage keys (has Owner/Contributor role)")
            except Exception as e:
                print(f"  ⚠️  Cannot list storage keys (might only have data plane access)")
                print(f"     This is OK if you have 'Storage Blob Data Contributor' role")
            
            # Get the storage account ID for role checking
            storage_account_id = account.id
            break
    else:
        print("⚠️  Storage account 'strategicaistuksdev02' not found in resource group")
        storage_account_id = None
        
except Exception as e:
    print(f"❌ Error accessing storage accounts: {str(e)}")
    print("   The Service Principal might lack permissions on the resource group")
    storage_account_id = None

🔍 Checking Storage Account Access...
----------------------------------------
✓ Found storage account: strategicaistuksdev02
  ✓ Can list storage keys (has Owner/Contributor role)


In [4]:
# ==============================================================================
# Check Role Assignments
# ==============================================================================
print("\n🔍 Checking Role Assignments...")
print("-"*40)

try:
    # Get role assignments for the Service Principal
    sp_object_id = None  # We need to get the object ID of the SP
    
    # List all role assignments in the resource group
    assignments = auth_client.role_assignments.list_for_resource_group(resource_group)
    
    sp_roles = []
    for assignment in assignments:
        # Check if this assignment is for our Service Principal
        # Note: This requires additional API calls to match the principal ID
        try:
            # Get role definition
            role_def = auth_client.role_definitions.get_by_id(assignment.role_definition_id)
            
            # You might need to match this with your SP's object ID
            # For now, we'll list all roles in the RG
            if assignment.principal_id:
                sp_roles.append({
                    'role': role_def.role_name,
                    'scope': assignment.scope,
                    'principal_id': assignment.principal_id
                })
        except:
            pass
    
    if sp_roles:
        print(f"Found {len(sp_roles)} role assignments in the resource group")
        # Note: Matching to specific SP requires additional API calls
        print("  (Full matching requires SP object ID lookup)")
    else:
        print("⚠️  No role assignments found (or insufficient permissions to view)")
        
except Exception as e:
    print(f"⚠️  Cannot list role assignments: {str(e)}")
    print("   This is normal if the SP only has limited permissions")


🔍 Checking Role Assignments...
----------------------------------------
Found 103 role assignments in the resource group
  (Full matching requires SP object ID lookup)
